# Importing Libraries

In [2]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.2/648.2 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.9 MB/s eta 0:00:00


In [3]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd

## Load the data

In [4]:
df_fraud = pd.read_csv('/content/fraud_data_merged.csv')

In [5]:
# drop signup_time, purchase_time
df_fraud.drop(columns=['signup_time', 'purchase_time'], inplace=True)
# drop country
df_fraud.drop(columns=['country'], inplace=True)

In [6]:
# drop na for fraud data
df_fraud.dropna(inplace=True)

In [9]:
# set target feature
X_fraud = df_fraud.drop(columns=['class'])
y_fraud = df_fraud['class']

In [10]:
# Split the data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

In [12]:
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_fraud_res, y_train_fraud_res = smote.fit_resample(X_train_fraud, y_train_fraud)

# Start MLflow experiment
mlflow.set_experiment("E-Commerce Fraud Detection")


2025/02/13 14:12:58 INFO mlflow.tracking.fluent: Experiment with name 'E-Commerce Fraud Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/539113993253351231', creation_time=1739455978769, experiment_id='539113993253351231', last_update_time=1739455978769, lifecycle_stage='active', name='E-Commerce Fraud Detection', tags={}>

# MLflow

In [13]:
# Start MLflow experiment
mlflow.set_experiment("E-Commerce Fraud Detection")

with mlflow.start_run():
    # Log parameters
    mlflow.log_param("dataset", "Fraud_Data")
    mlflow.log_param("model_type", "Random Forest")
    mlflow.log_param("class_imbalance_handling", "SMOTE")

    # Train the model
    rf_model_fraud = RandomForestClassifier(random_state=42)
    rf_model_fraud.fit(X_train_fraud_res, y_train_fraud_res)

    # Evaluate the model
    y_pred_fraud = rf_model_fraud.predict(X_test_fraud)
    report_fraud = classification_report(y_test_fraud, y_pred_fraud, output_dict=True)
    roc_auc_fraud = roc_auc_score(y_test_fraud, y_pred_fraud)


In [15]:
# Log metrics
mlflow.log_metric("precision", report_fraud["1"]["precision"])
mlflow.log_metric("recall", report_fraud["1"]["recall"])
mlflow.log_metric("f1_score", report_fraud["1"]["f1-score"])
mlflow.log_metric("roc_auc", roc_auc_fraud)

# Log the model
mlflow.sklearn.log_model(rf_model_fraud, "random_forest_fraud_data_model")
print("Fraud Data Model Logged to MLflow!")

2025/02/13 14:15:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Fraud Data Model Logged to MLflow!


In [18]:
# download the model as a pkl
import joblib
joblib.dump(rf_model_fraud, 'models/random_forest_fraud_data_model.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'models/random_forest_fraud_data_model.pkl'

In [ ]:
#### to view mlflow experiment use bash and
# mlflow ui

# Versioning Models

In [16]:
import mlflow.sklearn
# Load the logged model for fraud data
model_fraud = mlflow.sklearn.load_model("runs:/<RUN_ID>/random_forest_fraud_data_model")

MlflowException: Invalid value "<RUN_ID>" for parameter 'run_id' supplied.

# Model Explainability

In [17]:
pip install shap lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=aacfa4fe8daec8e09226c8709053c501c0e53ce8212fe8feedf9a6cc41d763bc
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


## Explaining the Random Forest Model with SHAP

In [ ]:
import shap

# Load the trained Random Forest model (replace with your model)
rf_model_fraud = joblib.load("models/random_forest_fraud_data_model.pkl")

# Initialize SHAP explainer for fraud data
explainer_fraud = shap.TreeExplainer(rf_model_fraud)
shap_values_fraud = explainer_fraud.shap_values(X_test_fraud)

In [ ]:
# Summary plot for fraud data
shap.summary_plot(shap_values_fraud, X_test_fraud, plot_type="bar")

In [ ]:
# The force plot visualizes the contribution of features for a single prediction
# Force plot for fraud data (first instance in the test set)
shap.force_plot(explainer_fraud.expected_value[1], shap_values_fraud[1][0, :], X_test_fraud.iloc[0, :])

In [ ]:
# The dependence plot shows the relationship between a feature and the model's output.
# Dependence plot for fraud data (e.g., feature 'purchase_value')
shap.dependence_plot("purchase_value", shap_values_fraud[1], X_test_fraud)

## Using LIME for Explainability

In [ ]:
import lime
import lime.lime_tabular

# Initialize LIME explainer for fraud data
explainer_fraud_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_fraud.values,
    feature_names=X_train_fraud.columns,
    class_names=['Non-Fraud', 'Fraud'],
    mode='classification'
)

In [ ]:
# Explain a single prediction for fraud data (first instance in the test set)
exp_fraud = explainer_fraud_lime.explain_instance(X_test_fraud.iloc[0, :], rf_model_fraud.predict_proba, num_features=10)
exp_fraud.show_in_notebook()